In [ ]:
# Use in the google colab to connect the google cloud in order to get the dataset
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
··········


In [ ]:
# Use in google colab in order to get information about GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Mon May 31 06:55:10 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Use in google colab in order to get the root path
!mkdir -p drive
!google-drive-ocamlfuse -o nonempty drive
import os
import sys
os.chdir('drive')

In [ ]:
import nltk
from functools import lru_cache
import re
import string
import numpy as np
import pandas as pd
import torch
from collections import Counter, defaultdict
from sklearn.preprocessing import OneHotEncoder
import torch.utils.data as Data
import torch.nn.functional as F
import torch
import torch.nn as nn
from torch.autograd import Variable
import os
import glob
from sklearn.utils import shuffle
import copy

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    # print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print(torch.cuda.device_count())
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

1
Device name: Tesla P100-PCIE-16GB


In [ ]:
class Modifiers:
    def __init__(self):
        C_INCR = 0.733
        B_INCR = 0.293
        B_DECR = -0.293

        N_SCALAR = -0.74
        self.NEGATE = \
            ["aint", "arent", "cannot", "cant", "couldnt", "darent", "didnt", "doesnt",
             "ain't", "aren't", "can't", "couldn't", "daren't", "didn't", "doesn't",
             "dont", "hadnt", "hasnt", "havent", "isnt", "mightnt", "mustnt", "neither",
             "don't", "hadn't", "hasn't", "haven't", "isn't", "mightn't", "mustn't",
             "neednt", "needn't", "never", "none", "nope", "nor", "not", "nothing", "nowhere",
             "oughtnt", "shant", "shouldnt", "uhuh", "wasnt", "werent",
             "oughtn't", "shan't", "shouldn't", "uh-uh", "wasn't", "weren't",
             "without", "wont", "wouldnt", "won't", "wouldn't", "rarely", "seldom", "despite"]

        # booster/dampener 'intensifiers' or 'degree adverbs'
        # 

        self.BOOSTER_DICT = \
            {"absolutely": B_INCR, "amazingly": B_INCR, "awfully": B_INCR, "completely": B_INCR, "considerably": B_INCR,
             "decidedly": B_INCR, "deeply": B_INCR, "effing": B_INCR, "enormously": B_INCR,
             "entirely": B_INCR, "especially": B_INCR, "exceptionally": B_INCR, "extremely": B_INCR,
             "fabulously": B_INCR, "flipping": B_INCR, "flippin": B_INCR,
             "fricking": B_INCR, "frickin": B_INCR, "frigging": B_INCR, "friggin": B_INCR, "fully": B_INCR,
             "fucking": B_INCR,
             "greatly": B_INCR, "hella": B_INCR, "highly": B_INCR, "hugely": B_INCR, "incredibly": B_INCR,
             "intensely": B_INCR, "majorly": B_INCR, "more": B_INCR, "most": B_INCR, "particularly": B_INCR,
             "purely": B_INCR, "quite": B_INCR, "really": B_INCR, "remarkably": B_INCR,
             "so": B_INCR, "substantially": B_INCR,
             "thoroughly": B_INCR, "totally": B_INCR, "tremendously": B_INCR,
             "uber": B_INCR, "unbelievably": B_INCR, "unusually": B_INCR, "utterly": B_INCR,
             "very": B_INCR,
             "almost": B_DECR, "barely": B_DECR, "hardly": B_DECR, "just enough": B_DECR,
             "kind of": B_DECR, "kinda": B_DECR, "kindof": B_DECR, "kind-of": B_DECR,
             "less": B_DECR, "little": B_DECR, "marginally": B_DECR, "occasionally": B_DECR, "partly": B_DECR,
             "scarcely": B_DECR, "slightly": B_DECR, "somewhat": B_DECR,
             "sort of": B_DECR, "sorta": B_DECR, "sortof": B_DECR, "sort-of": B_DECR}


In [ ]:
class Preprocessor:
    def clean_text(self, text):
        # '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
        # make text lowercase
        text1 = text.lower()
        # remove square brackets
        text1 = re.sub('\[.*?\]', '', text1)
        #remove <>
        text1 = re.sub('<.*?>+', '', text1)
        text1 = re.sub('\(.*?\)', ' ', text1)
        text1 = re.sub('\{.*?\}', ' ', text1)
        # remove links
        text1 = re.sub('https?://\S+|www\.\S+', ' ', text1)
        # remove punctuation
        text1 = re.sub('[%s]' % re.escape(string.punctuation), '', text1)
        # remove \n
        # text = re.sub('\n', '', text)
        # remove numbers
        text = re.sub('\w*\d\w*', '', text)
        return text1

    def __init__(self):
        self.stem = lru_cache(maxsize=10000)(nltk.stem.SnowballStemmer('english').stem)
        # self.stopwords = stopwords.words('english')
        self.tokenize = nltk.tokenize.TreebankWordTokenizer().tokenize

    def __call__(self, text):
        text1 = self.clean_text(text)
        tokens = self.tokenize(text1)
        # tokens = [token for token in tokens if token not in self.stopwords]
        # tokens = [self.stem(token) for token in tokens]

        return tokens

In [ ]:

class DataLoader:
    def __init__(self, data, batch_size, k, context_window = 1, preprocessor=Preprocessor(), enc_tokens = OneHotEncoder(), enc_modifiers = OneHotEncoder(), modifier = Modifiers(), dev = 'cpu'):
        self.df = data
        self.dev = dev
        self.batch_size = batch_size
        self.context_window = context_window
        self.modifiers = modifier.BOOSTER_DICT.keys()
        self.enc_modifiers = enc_modifiers
        # self.vocab_frequency = 5
        self.padding_length = 70
        self.fold_num = 10
        # self.first_sentence = 128
        # self.second_sentence = self.padding_length - self.first_sentence
        self.count = 0


        self.apply_preprocessor(preprocessor)
        enc_tokens.fit(self.vocab)
        self.enc_modifiers = enc_modifiers
        self.enc_modifiers.fit(self.modifier_id_to_fit)
        self.split(0.9, 0.1)

        train_num = len(self.train)
        if train_num % self.batch_size == 0:
            self.no_batch = train_num / self.batch_size
        else:
            self.no_batch = int(train_num / self.batch_size) + 1

        trainX, trainY = self.build_training_data(enc_tokens, self.train)
        testX, testY = self.build_training_data(enc_tokens, self.test)
        print(f'Train Dataset Shape : {trainX.shape}')
        print(f'Test Dataset Shape : {testX.shape}')
        print(f'Next Modifier : {self.count}')
        self.train_dataset = self.get_batch(trainX, trainY)
        self.test_dataset = self.get_batch(testX, testY)



    def get_batch(self, X, Y):
        dataset = Data.TensorDataset(X, Y)
        loader = Data.DataLoader(
            dataset=dataset,
            batch_size=self.batch_size,
            shuffle=True,
            # num_workers=2,
        )
        return loader


    def remove_low_frequency(self, list):
        new_list = []
        for x in list:
            if x in self.token_to_count.keys():
                new_list.append(x)
        return new_list


    def apply_preprocessor(self, preprocessor):
        self.df['tokens'] = [preprocessor(s) for s in self.df['sentence']]
        self.df['tokens'] = [x[:self.padding_length]  if len(x) > self.padding_length else x for x in self.df['tokens']]

        # for index, row in self.df.iterrows():
        #     if len(row['tokens']) > self.max_length:
        #         row[toke]

        ######## Changed part for negatives ###########
        self.token_to_count = Counter([x for l in self.df['tokens'] for x in l if x not in self.modifiers])
        self.modifiers_to_count = Counter([x for l in self.df['tokens'] for x in l if x in self.modifiers])
        ##############      End       #################

        # tmp_token_to_count = self.token_to_count.copy()
        # for index, value in tmp_token_to_count.items():
        #     if value <= self.vocab_frequency:
        #         self.token_to_count.pop(index)
        # self.df['tokens'] = [self.remove_low_frequency(x) for x in self.df['tokens']]
        self.max_length = self.get_max_length()
        print(f'Max Length : {self.max_length}')

        ######## Changed part for negatives ###########
        self.vocab = list([[term] for term in self.token_to_count.keys()])
        self.modifier_vocab = list([[term] for term in self.modifiers_to_count.keys()])
        self.modifier_token_to_id = {self.modifier_vocab[i][0]: i + 1 for i in range(len(self.modifier_vocab))}
        self.modifier_id_to_fit = list([[term] for term in self.modifier_token_to_id.values()])
        ##############      End       #################

        print(f'Vocab Size : {len(self.vocab)}')
        print(f'Modifier Size : {len(self.modifier_vocab)}')
        # print(self.token_to_count)

    def get_max_length(self):
        max_length = 0
        for index, row in self.df.iterrows():
            ######## Changed part for negatives ###########
            token_list = [x for x in row['tokens'] if x not in self.modifiers]
            ##############      End       #################
            tmp_length = len(token_list)
            if tmp_length > max_length:
                max_length = tmp_length
        return max_length

    def k_fold_partition(self, fold_num = 10):
        batch_size = int(len(self.train) / fold_num) # the number of data for each fold
        remain_num = len(self.train) - batch_size * fold_num # the remain data after partition
        self.fold_data = []
        fold_batch_list = [] # Average the remaining data to the folds
        for fold in range(fold_num):
          if remain_num > 0:
            remain_num -= 1
            fold_batch_list.append(batch_size + 1)
          else:
            fold_batch_list.append(batch_size)
        fold_index = 0 # The starting position of each division data
        for fold in range(fold_num):
          fold_texts = [fold_index, fold_index + fold_batch_list[fold]]
          self.fold_data.append(fold_texts)
          # print(f'fold_data : {fold_batch_list[fold]}')
          # print(f'fold_data type : {type(fold_batch_list[fold])}')
          # print(f'index type : {type(fold_index)}')
          fold_index = fold_index + fold_batch_list[fold]


    def k_fold_split(self, k):
        print(f'K : {k}, fold_data[k] : {self.fold_data[k]}')
        validation = self.train.iloc[self.fold_data[k][0]: self.fold_data[k][1]]
        train = self.train.iloc[0: self.fold_data[k][0]].append(self.train.iloc[self.fold_data[k][1]:])
        train_num = len(self.train) - (self.fold_data[k][1] - self.fold_data[k][0])
        if train_num % self.batch_size == 0:
            self.no_batch = train_num / self.batch_size
        else:
            self.no_batch = int(train_num / self.batch_size) + 1
        print(f' index : {self.no_batch}')
        return train, validation



    def split(self, train, test):
        index = int(train * len(self.df))
        self.train = self.df.iloc[0:index]
        self.test = self.df.iloc[index:]
        # if self.train_index % self.batch_size == 0:
        #     self.no_batch = self.train_index / self.batch_size
        # else:
        #     self.no_batch = int(self.train_index / self.batch_size) + 1
        # print(f' index : {self.no_batch}')

    def build_training_data(self, enc, df):
        X = []
        Y = []
        for index, row in df.iterrows():
            # build modifiers
            np_modifier = np.zeros(len(row['tokens']))
            for index, value in enumerate(row['tokens']):
                # label the word influenced by modifiers
                if value in self.modifiers:
                    id = self.modifier_token_to_id[value]
                    np_modifier[index] = -1
                    for i in range(1, self.context_window + 1):
                        if index - i > 0:
                            np_modifier[index - i] = id
                            if row['tokens'][index - i] in self.modifiers:
                              self.count += 1
                        if index + i < len(row['tokens']):
                            np_modifier[index + i] = id
                            if row['tokens'][index + i] in self.modifiers:
                              self.count += 1
                    # np_modifier[index - self.context_window] = id
                    # np_modifier[index] = -1
                    # if (index + self.context_window) < len(row['tokens']):
                    #     np_modifier[index + self.context_window] = id

            ######## Changed part for negatives ###########
            # delete modifier itself
            deleted_index = [i for i in range(len(np_modifier)) if np_modifier[i] == -1]
            np_modifier = np.delete(np_modifier, deleted_index)


            tmp = [enc.transform([[t]]).toarray()[0] for t in row['tokens'] if t not in self.modifiers]
            for i in range(len(tmp)):
                tmp[i] = np.append(tmp[i], np_modifier[i])
            if len(tmp) < self.max_length:
                pad_length = self.max_length - len(tmp)
                for i in range(pad_length):
                    tmp.append(np.append(np.zeros(len(self.vocab)), -1))
            ##############      End       #################
            X.append(tmp)
            Y.append(row['label'])
        X = np.array(X)
        Y = np.array(Y)
        X = torch.from_numpy(X).float()
        Y = torch.from_numpy(Y).long()
        print(f'Input Data Shape (sequence_num, sequence_len, vocab_size + 1) : {X.shape}')
        print(f'Input Label Shape : {Y.shape}')
        return X, Y


In [ ]:
class RelexNet(nn.Module):
    #Single step RNN.
    #input_size is char_vacab_size=26,hidden_size is number of hidden neurons，output_size is number of categories
    def __init__(self, vocabulary_size, modifier_size, len_seq, num_classes, enc):
        super(RelexNet, self).__init__()
        self.no_modifiers = np.zeros(modifier_size)
        self.num_classes = num_classes
        self.L = nn.Linear(vocabulary_size, num_classes)
        self.dropout = nn.Dropout(p=0.5)
        self.M = nn.Linear(modifier_size, 1).cuda()
        # self.B = nn.Linear(len_seq, len_seq)
        self.enc = enc
        self.softmax = nn.Softmax(dim=1)


    def handle_modifier(self, modifier_ids, batch):
        # modifier_id = modifier_ids.cpu()
        modifier = []
        influence_flag = []
        add = 0.5 * Variable(torch.ones(batch, 1))
        for i in range(batch):
            if modifier_ids[i] == 0:
                modifier.append(self.no_modifiers)
                influence_flag.append(0)
                add[i] = 1
            elif modifier_ids[i] == -1:
                modifier.append(self.no_modifiers)
                influence_flag.append(0)
                add[i] = 0
            else:
                # print(modifier_ids[i])
                modifier.append(self.enc.transform([[modifier_ids[i].cpu()]]).toarray()[0])
                influence_flag.append(1)
        modifier = torch.tensor(modifier).float().cuda()
        influence_flag = torch.tensor(influence_flag).float()
        influence_flag = influence_flag.reshape(-1, 1).cuda()
        add = add.cuda()
        return modifier, influence_flag, add



    def forward(self, input, B_layer):
        # X.shape = (batch, seq_len, vocab_size)
        T = input.shape[1]
        batch = input.shape[0]
        predict_y = Variable(torch.zeros(batch, self.num_classes))

        if B_layer is None:
            B_layer = Variable(torch.zeros(batch, self.num_classes)).cuda()

        for t in range(T):
            tmp = input[:, t, :-1]
            modifier_ids = input[:, t, -1]
            modifier, influence_flag, add = self.handle_modifier(modifier_ids, batch)

            L_onestep = self.L(tmp)
            L_onestep = self.dropout(L_onestep)
            # L_onestep = torch.sigmoid(L_onestep)
            # L_onestep = MyReLU_PLUS.apply(L_onestep)
            L_onestep = F.relu6(L_onestep)
            modifier_score = torch.sigmoid(self.M(modifier))
            modifier_score = modifier_score * influence_flag
            modifier_score = modifier_score + add
            # L_onestep = L_onestep + modifier_score

            L_onestep = L_onestep * modifier_score
            # L_onestep = L_onestep * negative_score

            B_layer = torch.add(B_layer, L_onestep)
            # print(B_layer)

            if self.num_classes == 1:
                predict_y[t] = F.sigmoid(B_layer)
            else:
                predict_y = self.softmax(B_layer)

        return predict_y, B_layer


In [ ]:
def train_model(dataset, model, optimizer, scheduler, num_epochs, dev):
    losses = []
    for epoch in range(num_epochs):
        # training mode
        # dataset.set_partition(dataset.train)
        model.train()
        total_train_loss = 0
        total_train_correct = 0
        count = 0
        for x, y in dataset.train_dataset:
            # for every batch in the training dataset perform one update step of the optimizer.
            state = None
            model.zero_grad()
            y_h, state = model(x.to(dev), state)
            loss = F.cross_entropy(y_h, y.to(dev))
            optimizer.zero_grad()
            # scheduler.zero_grad()
            loss.backward()
            optimizer.step()
            # print('{} optim: {}'.format(epoch, optimizer.param_groups[0]['lr']))
            scheduler.step()
            # print('{} scheduler: {}'.format(epoch, scheduler.get_lr()[0]))
            total_train_loss += loss.item()
            total_train_correct += (y_h.argmax(-1) == y.cuda()).float().mean()
            count += 1
        average_train_loss = total_train_loss / count
        average_train_accuracy = total_train_correct / count
        print('{} optim: {}'.format(epoch + 1, optimizer.param_groups[0]['lr']))
        # print('{} optim: {}'.format(epoch, optimizer.param_groups[0]['lr']))
        # print('{} scheduler: {}'.format(epoch, scheduler.get_lr()[0]))
        losses.append(average_train_loss)

        print(f'epoch {epoch + 1} accuracies: \t train: {average_train_accuracy}\t loss: {average_train_loss}\t')
        # print(f'epoch {epoch + 1} accuracies: \t train: {average_train_accuracy}\t valid: {average_valid_accuracy}\t valid loss: {average_valid_loss}\t precision: {average_precision}\t recall: {average_recall}\t')
        # dataset.shuffle()

    # test mode
    # dataset.set_partition(dataset.test)
    model.eval()
    total_test_correct = 0
    # total_test_tp = 0
    # total_test_fp = 0
    # total_test_fn = 0
    count = 0
    for x, y in dataset.test_dataset:
        state = None
        y_h, state = model(x.to(dev), state)
        total_test_correct += (y_h.argmax(-1) == y.cuda()).float().mean()
        # total_test_tp += float((y_h.argmax(-1) == y.cuda() & (y.cuda() == 1)).float())
        # total_test_fp += float((y_h.argmax(-1) != y.cuda() & (y.cuda() == 0)).float())
        # total_test_fn += float((y_h.argmax(-1) != y.cuda() & (y.cuda() == 1)).float())
        count += 1
    average_test_accuracy = total_test_correct / count
    # average_precision = total_test_tp / (total_test_tp + total_test_fp)
    # average_recall = total_test_tp / (total_test_tp + total_test_fn)

    # print(f'test accuracy {average_test_accuracy} precision {average_precision} recall {average_recall}')
    print(f'test accuracy {average_test_accuracy}')

    return losses, (average_train_accuracy, average_test_accuracy)

In [ ]:
num_epochs = 20
batch_size = 32  # Number of hidden neurons in model
context_window = 1
k = 9

dev = 'cuda' if torch.cuda.is_available() else 'cpu'  # If you have a GPU installed, use that, otherwise CPU
print(dev)
print('Loading data...')
all_files = glob.glob("data/sentiment/*.csv")
# all_files = glob.glob("*.csv")
data = pd.concat((pd.read_csv(f, header=None, index_col=None) for f in all_files))
# path = os.path.join('data', 'sentiment', 'amazon_cells_labelled.csv')
# data = pd.read_csv(path)
data.columns = ['sentence', 'label']
data = data.sample(frac=1, random_state=1)
# data = shuffle(data)
dataset = DataLoader(data, k=k, batch_size = batch_size, context_window=context_window, dev=dev)
# print(f'Data Size: {dataset.df.size()}')
print("Data Ready!")
vocabulary_size = len(dataset.vocab)
len_seq = dataset.max_length
num_classes = 2

# model = FastText(len(dataset.token_to_id)+2, num_hidden, len(dataset.class_to_id)).to(dev)
model = RelexNet(vocabulary_size=vocabulary_size, enc=dataset.enc_modifiers, modifier_size=len(dataset.modifier_vocab),len_seq=len_seq, num_classes=num_classes).cuda()
# optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(),lr=0.01,betas=(0.9,0.99))
# optimizer = torch.optim.Adam(model.parameters(),lr=0.01,betas=(0.9,0.99), weight_decay=10)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,milestones=[8 * dataset.no_batch, 14 * dataset.no_batch],gamma = 0.1, last_epoch=-1)

losses, accuracies = train_model(dataset, model, optimizer, scheduler, num_epochs, dev=dev)
print(losses)
# torch.save(model, os.path.join('./relexnet_model/classifier_M9.pth'))

cuda
Loading data...
Max Length : 68
Vocab Size : 5275
Modifier Size : 34
Input Data Shape (sequence_num, sequence_len, vocab_size + 1) : torch.Size([2700, 68, 5276])
Input Label Shape : torch.Size([2700])
Input Data Shape (sequence_num, sequence_len, vocab_size + 1) : torch.Size([300, 68, 5276])
Input Label Shape : torch.Size([300])
Train Dataset Shape : torch.Size([2700, 68, 5276])
Test Dataset Shape : torch.Size([300, 68, 5276])
Next Modifier : 38
Data Ready!
1 optim: 0.01
epoch 1 accuracies: 	 train: 0.5841912031173706	 loss: 0.6729133816326366	
2 optim: 0.01
epoch 2 accuracies: 	 train: 0.7332108020782471	 loss: 0.5983270813437069	
3 optim: 0.01
epoch 3 accuracies: 	 train: 0.7901960611343384	 loss: 0.5491945196600522	
4 optim: 0.01
epoch 4 accuracies: 	 train: 0.8089460730552673	 loss: 0.5244633271413691	
5 optim: 0.01
epoch 5 accuracies: 	 train: 0.8251225352287292	 loss: 0.5006516621393315	
6 optim: 0.01
epoch 6 accuracies: 	 train: 0.8327205777168274	 loss: 0.49386078540016626